# SQL Subqueries - Lab

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## CRM Database Schema

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="images/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database **data.sqlite**.

In [2]:
#Your code here; import the necessary packages
import sqlite3
import pandas as pd
conn = sqlite3.connect('data.sqlite')
c = conn.cursor()

## Select the Average Number of Orders on a Product by Product Basis

In [4]:
#Your code here; use a subquery. No join will be necessary.
c.execute("""SELECT AVG(numOrders)
            FROM (SELECT productCode, count(distinct orderNumber) as numOrders
                    FROM products
                    JOIN orderdetails using(productCode)
                    GROUP BY productCode)
                    ;""")
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
print('Number of results:', len(df))
df.head()

Number of results: 1


,AVG(numOrders)
0,27.486239


## Write an Equivalent Query using a Subquery

```SQL
select customerNumber,
       contactLastName,
       contactFirstName
       from customers
       join orders 
       using(customerNumber)
       where orderDate = '2003-01-31';
```

In [6]:
#Your code here; use a subquery. No join will be necessary.
c.execute("""SELECT customerNumber, contactLastName, contactFirstName
            FROM customers
            WHERE customerNumber IN (SELECT DISTINCT customerNumber 
                                    FROM orders WHERE orderDate = '2003-01-31');""")
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
print('Number of results:', len(df))
df.head()

Number of results: 1


,customerNumber,contactLastName,contactFirstName
0,141,Freyre,Diego


## Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

In [10]:
#Your code here
c.execute("""SELECT productCode, count(orderNumber) as num_orders, sum(quantityOrdered) as total_sold
            FROM orderdetails
            WHERE productCode 
            IN (SELECT productCode
                FROM products)
            GROUP BY productCode
            ORDER BY total_sold DESC;""")
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
print('Number of results:', len(df))
df.head()

Number of results: 109


,productCode,num_orders,total_sold
0,S18_3232,53,1808
1,S18_1342,28,1111
2,S700_4002,28,1085
3,S18_3856,28,1076
4,S50_1341,28,1074


## Select the Product Name and the  Total Number of People Who Have Ordered Each Product

Sort the results in descending order.

In [11]:
#Your code here
c.execute("""SELECT productName, count(distinct customerNumber) as num_people_order
            FROM products
            JOIN orderdetails using(productCode)
            JOIN orders using(orderNumber)
            GROUP BY productCode
            ORDER BY num_people_order DESC;""")
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
print('Number of results:', len(df))
df.head()

Number of results: 109


,productName,num_people_order
0,1992 Ferrari 360 Spider red,40
1,1952 Alpine Renault 1300,27
2,1972 Alfa Romeo GTA,27
3,1934 Ford V8 Coupe,27
4,Boeing X-32A JSF,27


## Select the Employee Number, Office Code, City (of the office), and Name (First and Last) of those Employees who Sold Products that Have Been Ordered by Less Then 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [20]:
#Your code here
c.execute("""SELECT distinct employeeNumber, officeCode, offices.city, firstName, lastName
            FROM offices
            JOIN employees using(officeCode)
            JOIN customers ON employees.employeeNumber = customers.salesRepEmployeeNumber
            JOIN orders using(customerNumber)
            JOIN orderdetails using(orderNumber)
            JOIN products using(productCode)
            WHERE productCode IN
                (SELECT productCode
                FROM products
                JOIN orderdetails using(productCode)
                JOIN orders using(orderNumber)
                GROUP BY productCode
                HAVING count(distinct customerNumber) < 20)
                ;""")
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
print('Number of results:', len(df))
df.head(20)

Number of results: 15


,employeeNumber,officeCode,city,firstName,lastName
0,1370,4,Paris,Gerard,Hernandez
1,1501,7,London,Larry,Bott
2,1337,4,Paris,Loui,Bondur
3,1166,1,San Francisco,Leslie,Thompson
4,1286,3,NYC,Foon Yue,Tseng
5,1612,6,Sydney,Peter,Marsh
6,1611,6,Sydney,Andy,Fixter
7,1401,4,Paris,Pamela,Castillo
8,1621,5,Tokyo,Mami,Nishi
9,1323,3,NYC,George,Vanauf


## Select the Employee Number, Name (First and Last) and Number of Customers of Employees Who's Customers Have an Average Credit Limit of Over 15K

In [ ]:
#Your code here
c.execute("""SELECT distinct employeeNumber, firstName, lastName, count(distinct customerNumber) as num_customer
            FROM employees e
            JOIN customers c ON e.employeeNumber = c.salesRepEmployeeNumber
            GROUP BY 1, 2, 3
            HAVING ;""")
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
print('Number of results:', len(df))
df.head()

## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!